# Key moments extraction

In [1]:
import json

from pathlib import Path

from video_intelligence.utils import io
from video_intelligence.config import Config
from video_intelligence.export.bq import BigQueryExporter
from video_intelligence.processing.sequences import SequenceProcessor

In [ ]:
config_file = str(Path.cwd().parent / "config.yaml")
config = Config.from_yaml(config_file)

In [3]:
gcs_io = io.GcsIO(config.project_id, f"gs://{config.bucket}")
local_io = io.LocalIO(root_path=config.paths.tmp)

In [4]:
videos = gcs_io.list_files(path=config.preprocessed_path, rec=True)

In [5]:
processor = SequenceProcessor(config)

In [6]:
results = {}

for gcs_path in videos:
    video_bytes = gcs_io.get_video(path=gcs_path)
    detections = processor._get_detections_from_bq(gcs_path)
    response = processor.extract_sequences(
        video_bytes, detections
    )
    results[gcs_path] = response

In [7]:
with open(config.paths.sequences, "w") as f:
    f.write(json.dumps(results, indent=2))

In [8]:
exporter = BigQueryExporter(config)
exporter.export_sequences(config.paths.results)

Loading sequences data from /usr/local/google/home/gagata/code/video-intelligence/results/results.json
Prepared sequences for preprocessed/0000f77c-6257be58/0000f77c-6257be58.mp4
Prepared sequences for preprocessed/00225f53-67614580/00225f53-67614580.mp4
Prepared sequences for preprocessed/00091078-7cff8ea6/00091078-7cff8ea6.mp4
Loading 3 updated records to BigQuery...


BadRequest: 400 Error while reading data, error message: JSON table encountered too many errors, giving up. Rows: 1; errors: 1. Please look into the errors[] collection for more details.; reason: invalid, message: Error while reading data, error message: JSON table encountered too many errors, giving up. Rows: 1; errors: 1. Please look into the errors[] collection for more details.; reason: invalid, message: Error while reading data, error message: JSON processing encountered too many errors, giving up. Rows: 1; errors: 1; max bad: 0; error percent: 0; reason: invalid, message: Error while reading data, error message: JSON parsing error in row starting at position 0: No such field: sequences.timestamp.